1.Import packages

In [11]:
#dirty soln, but enables importing zfel to work.
import sys, os
parentdir = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
sys.path.append(parentdir) if parentdir not in sys.path else sys.path
print(sys.path)

['/Users/Sathya/anaconda3/envs/pyrecent/lib/python38.zip', '/Users/Sathya/anaconda3/envs/pyrecent/lib/python3.8', '/Users/Sathya/anaconda3/envs/pyrecent/lib/python3.8/lib-dynload', '', '/Users/Sathya/anaconda3/envs/pyrecent/lib/python3.8/site-packages', '/Users/Sathya/anaconda3/envs/pyrecent/lib/python3.8/site-packages/IPython/extensions', '/Users/Sathya/.ipython', '/Users/Sathya/Documents/SLAC/zfel']


In [13]:
import numpy as np
from numpy import genfromtxt
import scipy
from scipy import special
from zfel import general_load_bucket
import matplotlib.pyplot as plt 

2.Input parameters

In [16]:
Nruns=1                             # Number of runs
npart   = 512                       # n-macro-particles per bucket 
s_steps = 200#31                    # n-sample points along bunch length
z_steps = 200#20                    # n-sample points along undulator
energy  = 4313.34*1E6               # electron energy [eV]
eSpread = 0#1.0e-4                  # relative rms energy spread [ ]
emitN   = 1.2e-6                    # normalized transverse emittance [m-rad]
currentMax = 3900                   # peak current [Ampere]
beta = 26                           # mean beta [meter]
unduPeriod = 0.03                   # undulator period [meter]
unduK = 3.5                         # undulator parameter, K [ ]
unduL = 70#30                       # length of undulator [meter]
radWavelength = 1.5e-9              # seed wavelength? [meter], used only in single-freuqency runs
dEdz = 0                            # rate of relative energy gain or taper [keV/m], optimal~130
iopt = 5                            # 5=SASE, 4=seeded
P0 = 10000*0.0                      # small seed input power [W]
constseed = 1                       # whether we want to use constant random seed for reproducibility, 1 Yes, 0 No
particle_position=genfromtxt('./Inputs/weird_particle_position.csv', delimiter=',') # or None  
# particle information with positions in meter and eta,\
# if we want to load random particle positions and energy, then set None
hist_rule='square-root'             # 'square-root' or 'sturges' or 'rice-rule' or 'self-design', number \
                                    #  of intervals to generate the histogram of eta value in a bucket

3. Calculating intermediate parameters and bunching factor

In [33]:
# whether to use constant random seed for reproducibility
if constseed==1:
    np.random.seed(22)

# Some constant values
mc2 = 0.51099906E6#510.99906E-3      # Electron rest mass in eV
c = 2.99792458E8        # light speed in meter
e = 1.60217733E-19      # electron charge in Coulomb

gamma0  = energy/mc2                                    # central energy of the beam in unit of mc2

resWavelength = unduPeriod*(1+unduK**2/2.0)\
                /(2*gamma0**2)                          # resonant wavelength
coopLength = resWavelength/unduPeriod                # cooperation length
gainLength = 1                                      # rough gain length
#cs0  = bunchLength/coopLength                           # bunch length in units of cooperation length     
z0    = unduL/gainLength                                # wiggler length in units of gain length
delt  = z0/z_steps                                      # integration step in z0 ~ 0.1 gain length
dels  = delt                                            # integration step in s0 must be same as in z0 
gbar  = (resWavelength-radWavelength)\
        /(radWavelength)                                # scaled detune parameter
delg  = eSpread                                         # Gaussian energy spread in units of rho 
Ns    = currentMax*unduL/unduPeriod/z_steps\
        *resWavelength/c/e                              # N electrons per s-slice [ ]
#load buckets
GLBResult=general_load_bucket.general_load_bucket(npart, Ns, coopLength, particle_position, \
                                                            s_steps, dels, hist_rule, gbar=gbar, \
                                                            delg=delg, iopt=iopt)
thet_init = GLBResult['thet_init']

bunching=np.mean(np.real(np.exp(-1j*thet_init)),axis=1)\
                          +np.mean(np.imag(np.exp(-1j*thet_init)),axis=1)*1j            #bunching factor calculation   

4. Verify whether the initial bunching level is reasonable

In [34]:
print(np.sqrt(np.mean(np.absolute(bunching)**2)))
print(1/np.sqrt(Ns))

0.0008998043866928951
0.0008389101946493842


In [30]:
print(bunching)

[ 4.01984714e-04+1.71370977e-04j -4.66663358e-04-1.94303485e-04j
  5.01625206e-04-1.18956616e-04j  3.78691335e-04+6.40077661e-04j
  1.13388115e-05+8.12275326e-05j  4.18544819e-04-9.82686433e-04j
 -3.61089111e-04+9.57741692e-04j -1.13933888e-03+2.71766911e-05j
  1.02673701e-04+6.10609989e-04j  5.74705123e-04+5.14904204e-04j
 -1.50366211e-05+1.10202806e-04j -1.14180994e-04+3.05971024e-05j
  1.04139789e-03+2.79988241e-04j  4.81343410e-05-2.04222563e-04j
  7.00988605e-04+3.51101664e-04j -5.26823328e-04+2.49805577e-04j
 -1.04116751e-04-1.53628503e-03j -2.02266459e-03+5.17948185e-04j
  1.98705779e-04+1.15856569e-03j  9.33902704e-04-8.48183114e-04j
 -1.98333060e-03-4.93789773e-04j -7.38442876e-04-5.48435379e-04j
 -6.66004905e-04-1.53244311e-03j  3.44535725e-04-1.10730609e-04j
 -9.02690692e-04+5.28613766e-04j  2.75459156e-04+6.19822889e-04j
  7.95605323e-05+2.67027962e-04j -1.15325564e-03-2.76383741e-04j
 -1.05913778e-04-7.54250775e-04j  5.47293556e-04-8.19845850e-04j
  3.71310784e-04-1.373698